In [1]:
## To work with the data
import pandas as pd
import numpy as np

In [2]:
## Let's read our data
df = pd.read_csv("consumer_compliants.zip", compression = "zip")

In [3]:
## Take a loot at the data
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,NaN,NaN,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,NaN,CURO Intermediate Holdings,CO,806XX,NaN,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX an...",NaN,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,NaN,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in...",NaN,CAPITAL ONE FINANCIAL CORPORATION,CA,NaN,NaN,Consent provided,Web,3/6/2020,Closed with explanation,Yes,NaN,3556237
4,2/14/2020,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,This debt was incurred due to medical malpract...,Company believes it acted appropriately as aut...,"Merchants and Professional Bureau, Inc.",OH,432XX,NaN,Consent provided,Web,2/14/2020,Closed with explanation,Yes,NaN,3531704


In [4]:
## To be abe to read whole narrative
# pd.set_option("display.max_colwidth", -1)

ValueError: Value must be a nonnegative integer or None

- there are a lot of attributes which we are not interested in for the task in hand. We'll be working on Customer complaint narrative and try to perform topic modeling on that.

In [10]:
## Useful columns
df = df[['Product', 'Consumer complaint narrative']]
df.columns = ['Product', 'Narrative']

In [11]:
df.head()

,Product,Narrative
0,Vehicle loan or lease,This auto loan was opened on XX/XX/2020 in XXX...
1,Debt collection,In XXXX of 2019 I noticed a debt for {$620.00}...
2,Vehicle loan or lease,"As stated from Capital One, XXXX XX/XX/XXXX an..."
3,Checking or savings account,"Please see CFPB case XXXX. \n\nCapital One, in..."
4,Debt collection,This debt was incurred due to medical malpract...


In [12]:
df.shape

(57453, 2)

In [13]:
df['Product'].value_counts()
## Class imbalance

Product
Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: count, dtype: int64

#### Preprocessing

In [14]:
## performes tokenization.
from nltk import word_tokenize

In [15]:
## Custom Tokenization:
## Those card numbers with Xx in them are of no use. also, I am not interested in amount and other numerical features. 
def tokenize(sentance):
    tokens = word_tokenize(sentance)
    tokens = [word for word in tokens if ( word.isalpha() and len(word)>3 and len(word.strip('Xx/'))>2) ]
    return tokens

In [16]:
## Used for splitting our data into train and test set.
from sklearn.model_selection import train_test_split

X, y = df[['Narrative']], df[['Product']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

In [18]:
## Resolving class Imbalance
from imblearn.over_sampling import RandomOverSampler

strategy = {
    "Debt collection"              :  21772,
    "Credit card or prepaid card"  :  13193,
    "Mortgage"                     :  11000 ,
    "Checking or savings account"  :  9000 ,
    "Student loan"                 :  7000 ,
    "Vehicle loan or lease"        :  7000 }

ros = RandomOverSampler(sampling_strategy = strategy, random_state=42)
X_train, y_train = ros.fit_resample(X_train, y_train)

In [19]:
## we need to represent words as numerical values/ vectors.
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
## Very helpful indeed. Different libraries may have differet implementation of the same functionality.
## It is useful to take a look at the parameters of the function
TfidfVectorizer?

Init signature:
TfidfVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    analyzer='word',
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.float64'>,
    norm='l2',
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=False,
)
Docstring:     
Convert a collection of raw documents to a matrix of TF-IDF features.

Equivalent to :class:`CountVectorizer` followed by
:class:`TfidfTransformer`.

For an example of usage, see
:ref:`sphx_glr_auto_examples_text_plot_document_classification_20newsgroups.py`.

For an efficiency comparision of the different feature extractors, see
:ref:`sphx_glr_auto_examples_text_plot_hashing_vs_dict_vectorizer.py`.

Read more in the :ref:`User Guide <text_feature_extraction>`.

Paramete

In [21]:
vectorizer = TfidfVectorizer(tokenizer = tokenize, stop_words = 'english', 
                            max_df = 0.75, min_df = 50, max_features = 10000)
vectors = vectorizer.fit_transform(X_train['Narrative'])

c:\Users\raj\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [22]:
vectors.shape

(68965, 4943)

In [28]:
## Feature names mapped to the array index.
## 1st feature in vectors is the word at 1 index of the list.
vectorizer.get_feature_names_out()

array(['aadvantage', 'abandoned', 'abide', ..., 'youre', 'zero', 'zone'],
      dtype=object)

### Model Building

In [29]:
from sklearn.decomposition import LatentDirichletAllocation

In [30]:
LatentDirichletAllocation?

Init signature:
LatentDirichletAllocation(
    n_components=10,
    *,
    doc_topic_prior=None,
    topic_word_prior=None,
    learning_method='batch',
    learning_decay=0.7,
    learning_offset=10.0,
    max_iter=10,
    batch_size=128,
    evaluate_every=-1,
    total_samples=1000000.0,
    perp_tol=0.1,
    mean_change_tol=0.001,
    max_doc_update_iter=100,
    n_jobs=None,
    verbose=0,
    random_state=None,
)
Docstring:     
Latent Dirichlet Allocation with online variational Bayes algorithm.

The implementation is based on [1]_ and [2]_.

.. versionadded:: 0.17

Read more in the :ref:`User Guide <LatentDirichletAllocation>`.

Parameters
----------
n_components : int, default=10
    Number of topics.

    .. versionchanged:: 0.19
        ``n_topics`` was renamed to ``n_components``

doc_topic_prior : float, default=None
    Prior of document topic distribution `theta`. If the value is None,
    defaults to `1 / n_components`.
    In [1]_, this is called `alpha`.

topic_word_p

In [31]:
lda = LatentDirichletAllocation(n_components = 6, learning_method = 'online', max_iter = 100,
                         n_jobs = -1, random_state = 123)

W1 = lda.fit_transform(vectors)

H1 = lda.components_

In [32]:
## H1
W1

array([[0.03826295, 0.01854838, 0.03993724, 0.09456617, 0.01835607,
        0.79032918],
       [0.03284839, 0.03272191, 0.03276249, 0.03349488, 0.0333319 ,
        0.83484044],
       [0.02750916, 0.0478809 , 0.0276351 , 0.02772895, 0.02712774,
        0.84211816],
       ...,
       [0.03619558, 0.03749331, 0.03575796, 0.38915436, 0.03611963,
        0.46527916],
       [0.02123505, 0.02122891, 0.89386232, 0.02122752, 0.02122279,
        0.02122342],
       [0.25284468, 0.03635479, 0.03596977, 0.60240052, 0.0361438 ,
        0.03628645]])

In [33]:
## W1 -> Probability of document being of the topic(i). 6 topics->6 probabilities.
## H1 -> Topic-> Words
print(W1.shape, H1.shape)

(68965, 6) (6, 4943)


In [34]:
vocab = np.array(vectorizer.get_feature_names_out())

## Gives last (numwords-1) numbers in descending order. from n to 0.
## So, for each topic, we get top 10 words that appear in that topic.
def get_top_words(words):
    res = []
    for index in np.argsort(words)[:-10:-1]:
        res.append(vocab[index])
    return res
## top_words = lambda words: [vocab[index] for index in np.argsort(words)[:-top:-1]]

## Get topic words using H1.
topic_words = (get_top_words(words) for words in H1)
topics = [','.join(t) for t in topic_words]

In [37]:
topics
## These are based on my intuition. Can be interpreted entirely differently as well.
## 0 -> Student Loans
## 1 -> Mortgage
## 2 -> Credit card or prepaid card
## 3 -> Debt Collection
## 4 -> Vehicle loan or lease
## 5 -> Checking or savings account

['loan,loans,payments,student,navient,payment,forbearance,rate,income',
 'mortgage,escrow,loan,modification,property,insurance,home,foreclosure,servicing',
 'usaa,bonus,opened,promotion,checks,suntrust,union,credit,relation',
 'debt,credit,collection,company,account,report,information,letter,reporting',
 'identity,theft,victim,belong,report,affidavit,debt,santander,does',
 'account,bank,card,payment,credit,told,called,money,said']

In [38]:
colnames = ["Topic_" + str(i) for i in range(lda.n_components)]
df1 = pd.DataFrame(np.round(W1, 2), columns = colnames, index = X_train.index)
# significant_topic = np.argmax(df1.values, axis = 1)
df1["Dominant Topic"] = np.argmax(df1.values, axis = 1)

In [39]:
df1 = X_train.join(df1)
df1 = df1.join(y_train)

In [40]:
df1.head()

,Narrative,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Dominant Topic,Product
0,On ( date ) I contacted Bank of America in reg...,0.04,0.02,0.04,0.09,0.02,0.79,5,Checking or savings account
1,I had a issue with capital one buy power card ...,0.03,0.03,0.03,0.03,0.03,0.83,5,Credit card or prepaid card
2,"On XX/XX/2019, 20 fraudulent charges were made...",0.03,0.05,0.03,0.03,0.03,0.84,5,Credit card or prepaid card
3,Started a Credit Card through XXXX XXXX with X...,0.03,0.03,0.03,0.06,0.03,0.82,5,Credit card or prepaid card
4,I went in to a Union Bank located in XXXX XXXX...,0.02,0.02,0.07,0.08,0.02,0.79,5,Checking or savings account


In [41]:
W2 = lda.transform(vectorizer.transform(X_test['Narrative']))

X_test["Dominant Topic"] = np.argmax(W2, axis =1)

In [42]:
X_test.join(y_test).head(10)

,Narrative,Dominant Topic,Product
40496,Our mortgage loan is serviced by The Money Sou...,5,Mortgage
17199,To Whom It May Concern : This letter is regard...,3,Debt collection
45428,I started receiving calls from Portfolio Recov...,3,Debt collection
56085,"This company, Credence Resource Management is ...",3,Debt collection
41199,"I shopped at Nordstrom rack on XX/XX/XXXX, and...",5,Credit card or prepaid card
35599,In XXXX I filed complaint number XXXX. I also ...,5,Mortgage
17220,I recently started to look into my credit back...,3,Debt collection
14443,This company ( National credit Adjusters ) whi...,3,Debt collection
27947,I present the complaint of the collection agen...,3,Debt collection
172,I moved out of my old apartment complex before...,3,Debt collection
